In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import osmnx as ox
import networkx as nx
import random
import time
import pickle
from collections import deque

In [2]:
ox.config(log_console=False)
G = ox.graph_from_place("Lviv Oblast, Ukraine", network_type='drive')
G = ox.utils_graph.get_largest_component(G, strongly=True)

node_list = list(G.nodes)
node_id_map = {n: i for i, n in enumerate(node_list)}
node_lookup = {i: n for n, i in node_id_map.items()}

C:\Users\Ryzen\AppData\Local\Temp\ipykernel_14492\618133296.py:1: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  ox.config(log_console=False)
C:\Users\Ryzen\AppData\Local\Temp\ipykernel_14492\618133296.py:3: FutureWarning: The `get_largest_component` function is deprecated and will be removed in the v2.0.0 release. Replace it with `truncate.largest_component` instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  G = ox.utils_graph.get_largest_component(G, strongly=True)


In [3]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_dim=2, hidden_dim=128):
        super(PolicyNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, node_feats):
        return self.net(node_feats).squeeze(-1)

def get_node_coordinates(node_id):
    node = G.nodes[node_id]
    return torch.tensor([node['x'], node['y']], dtype=torch.float32)

def get_neighbors(node_id):
    return list(G.successors(node_id))

def distance(n1, n2):
    return ox.distance.euclidean_dist_vec(G.nodes[n1]['y'], G.nodes[n1]['x'],
                                          G.nodes[n2]['y'], G.nodes[n2]['x'])

def train_policy_network(model_path='policy_model.pt', num_episodes=5000):
    policy = PolicyNetwork()
    optimizer = optim.Adam(policy.parameters(), lr=1e-3)
    gamma = 0.99
    print_every = 100

    for episode in range(num_episodes):
        start_node = random.choice(node_list)
        route = [start_node]
        log_probs = []
        rewards = []
        
        for _ in range(random.randint(4, 10)):
            curr = route[-1]
            neighbors = get_neighbors(curr)
            if not neighbors:
                break

            feats = torch.stack([get_node_coordinates(n) for n in neighbors])
            logits = policy(feats)
            probs = torch.softmax(logits, dim=0)

            dist = torch.distributions.Categorical(probs)
            action = dist.sample()
            next_node = neighbors[action.item()]
            
            route.append(next_node)
            log_probs.append(dist.log_prob(action))
            rewards.append(-distance(curr, next_node))

        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)

        loss = 0
        for log_prob, R in zip(log_probs, returns):
            loss -= log_prob * R

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if episode % print_every == 0:
            print(f"Episode {episode}")

    torch.save(policy.state_dict(), model_path)
    return policy

def load_policy_network(model_path='policy_model.pt'):
    policy = PolicyNetwork()
    if os.path.exists(model_path):
        policy.load_state_dict(torch.load(model_path))
        print("Loaded saved model.")
    else:
        print("Training new model...")
        policy = train_policy_network(model_path)
    policy.eval()
    return policy

def generate_route(policy, lat_lon_points):
    path = []
    for i in range(len(lat_lon_points) - 1):
        start = ox.distance.nearest_nodes(G, lat_lon_points[i][1], lat_lon_points[i][0])
        end = ox.distance.nearest_nodes(G, lat_lon_points[i+1][1], lat_lon_points[i+1][0])

        current = start
        sub_path = [current]
        for _ in range(50): #!!!!!!!!!!!!!!!!!!!!!
            if current == end:
                break
            neighbors = get_neighbors(current)
            if not neighbors:
                break

            feats = torch.stack([get_node_coordinates(n) for n in neighbors])
            logits = policy(feats)
            probs = torch.softmax(logits, dim=0)
            action = torch.argmax(probs)
            next_node = neighbors[action.item()]
            sub_path.append(next_node)
            current = next_node

        path.extend(sub_path[:-1])
    path.append(end)
    return path

def route_creator(destinations):
    return [random.choice(map_points) for _ in range(destinations)]

In [4]:
map_points=[(49.80611220996008, 23.97955612862654),
(49.80532285209437, 23.98237781261442),
(49.80478275771747, 23.987291619711186),
(49.8042495817099, 23.99243073217024),
(49.80390336037501, 23.996432588052162),
(49.80395183150313, 23.999769256034998),
(49.80649303590949, 23.999114796958104),
(49.81293216592172, 24.000763128745433),
(49.81766865049101, 24.003570346218133),
(49.823736810282, 24.00771312858426),
(49.82582162024907, 24.00966317414233),
(49.832640701340466, 24.01859487887445),
(49.83680583050842, 24.02202637853643),
(49.840145382636926, 24.027186953530506),
(49.84012073274527, 24.028125056144603),
(49.839253501750214, 24.03253064738627)]

In [5]:
policy = load_policy_network()

Loaded saved model.


In [10]:
print(policy)

PolicyNetwork(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [6]:
import time

for stops in [2, 4, 8, 16, 64]:
    route = route_creator(stops)
    print(stops)
    
    timer_start = time.time()
    node_path = generate_route(policy, route)
    
    timer_end = time.time()
    time_total = timer_end - timer_start
    print("time to generate route: " + str(time_total) + " s")

2
time to generate route: 0.7916774749755859 s
4
time to generate route: 1.5070152282714844 s
8
time to generate route: 3.161550998687744 s
16
time to generate route: 7.145020008087158 s
64
time to generate route: 33.2262167930603 s


In [7]:
route = route_creator(2)
node_path = generate_route(policy, route)